In [1]:
!pip install joblib

In [2]:
!pip install scapy joblib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 21.5 MB/s eta 0:00:00


In [ ]:
#using csv file

In [5]:
import pandas as pd
import joblib
import time
import numpy as np

model = joblib.load("network_intrusion_model2.pkl")
scaler = joblib.load("scaler.pkl")

selected_features = [
    'PSH Flag Count',
    'Bwd Packets/s',
    'Init_Win_bytes_forward',
    'Fwd IAT Total',
    'URG Flag Count',
    'Flow Duration',
    'Bwd IAT Total',
    'ACK Flag Count',
    'Bwd Packet Length Min',
    'Min Packet Length'
]

df = pd.read_csv("balanced_network_data3.csv")
df = df[selected_features + ['Label']].dropna()




In [6]:
import warnings
warnings.filterwarnings('ignore')

df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)

for index, row in df_shuffled.iterrows():
    features = row[selected_features].values.reshape(1, -1)
    scaled_features = scaler.transform(features)
    prediction = model.predict(features)[0]

    print(f"Simulated Packet {index + 1}: Predicted = {prediction}, Actual = {row['Label']}")

    time.sleep(0.5)

#press i twice to stop the packet capture

Simulated Packet 1: Predicted = DDoS, Actual = DDoS


Simulated Packet 2: Predicted = PortScan, Actual = PortScan
Simulated Packet 3: Predicted = PortScan, Actual = PortScan
Simulated Packet 4: Predicted = SSH-Patator, Actual = SSH-Patator
Simulated Packet 5: Predicted = DDoS, Actual = DDoS
Simulated Packet 6: Predicted = SSH-Patator, Actual = SSH-Patator
Simulated Packet 7: Predicted = FTP-Patator, Actual = FTP-Patator
Simulated Packet 8: Predicted = FTP-Patator, Actual = FTP-Patator
Simulated Packet 9: Predicted = PortScan, Actual = PortScan
Simulated Packet 10: Predicted = DDoS, Actual = DDoS
Simulated Packet 11: Predicted = DDoS, Actual = DDoS
Simulated Packet 12: Predicted = BENIGN, Actual = BENIGN
Simulated Packet 13: Predicted = PortScan, Actual = PortScan
Simulated Packet 14: Predicted = BENIGN, Actual = BENIGN
Simulated Packet 15: Predicted = BENIGN, Actual = BENIGN
Simulated Packet 16: Predicted = BENIGN, Actual = BENIGN
Simulated Packet 17: Predicted = SSH-Patator, Actual = SSH-Patator
Simulated Packet 18: Predicted = SSH-Patat

In [1]:
import tkinter as tk
from tkinter import filedialog, scrolledtext
import pandas as pd
import joblib
import threading
import time
import numpy as np
from scapy.all import sniff, IP, TCP
import warnings
import sys
warnings.filterwarnings('ignore')

# Load model and scaler
model = joblib.load('network_intrusion_model2.pkl')
scaler = joblib.load('scaler.pkl')

selected_features = [
    'PSH Flag Count', 'Bwd Packets/s', 'Init_Win_bytes_forward',
    'Fwd IAT Total', 'URG Flag Count', 'Flow Duration',
    'Bwd IAT Total', 'ACK Flag Count', 'Bwd Packet Length Min', 'Min Packet Length'
]

flow_data = {"start_time": None, "packet_lengths": [], "iat_times": [], "packet_count": 0, "bwd_packet_count": 0}
stop_sniffing = False

# === Redirect print output to GUI ===
class TextRedirector:
    def __init__(self, widget):
        self.widget = widget

    def write(self, message):
        self.widget.insert(tk.END, message)
        self.widget.see(tk.END)

    def flush(self):
        pass

# === GUI Application ===
class AnomalyApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Real-Time Network Anomaly Detection")
        self.root.geometry("960x620")
        self.current_frame = None
        self.create_welcome_screen()

    def clear_frame(self):
        if self.current_frame:
            self.current_frame.destroy()

    def create_welcome_screen(self):
        self.clear_frame()
        frame = tk.Frame(self.root)
        frame.pack(expand=True)

        tk.Label(frame, text="Computer Networks Course", font=("Helvetica", 20)).pack(pady=10)
        tk.Label(frame, text="Real-Time Network Anomaly Detection Project", font=("Helvetica", 16)).pack(pady=5)
        tk.Label(frame, text="By 22K-4332 and 22K-4547", font=("Helvetica", 14)).pack(pady=5)
        tk.Button(frame, text="Proceed", font=("Helvetica", 14), command=self.create_simulation_screen).pack(pady=20)

        self.current_frame = frame

    def create_simulation_screen(self):
        self.clear_frame()
        frame = tk.Frame(self.root)
        frame.pack(fill=tk.BOTH, expand=True)

        tk.Label(frame, text="Model Test Data Simulation", font=("Helvetica", 16)).pack(pady=10)

        self.sim_output = scrolledtext.ScrolledText(frame, width=100, height=25)
        self.sim_output.pack(pady=10)

        # Redirect print to GUI
        sys.stdout = TextRedirector(self.sim_output)

        self.next_button = tk.Button(frame, text="Go to Real-Time Detection", command=self.create_realtime_screen)
        self.next_button.pack(pady=10)
        self.next_button.config(state=tk.DISABLED)

        self.current_frame = frame
        threading.Thread(target=self.run_simulation).start()

    def run_simulation(self):
        try:
            df = pd.read_csv("balanced_network_data3.csv")
            df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)

            for index, row in df_shuffled.iterrows():
                features = row[selected_features].values.reshape(1, -1)
                scaled_features = scaler.transform(features)
                prediction = model.predict(features)[0]
                actual = row['Label']

                print(f"Simulated Packet {index + 1}: Predicted = {prediction}, Actual = {actual}")
                time.sleep(0.5)

            self.next_button.config(state=tk.NORMAL)

        except Exception as e:
            print(f"Error: {str(e)}")

    def create_realtime_screen(self):
        self.clear_frame()
        frame = tk.Frame(self.root)
        frame.pack(fill=tk.BOTH, expand=True)

        tk.Label(frame, text="Real-Time Anomaly Detection", font=("Helvetica", 16)).pack(pady=10)

        self.real_output = scrolledtext.ScrolledText(frame, width=100, height=25)
        self.real_output.pack(pady=10)
        self.real_output.tag_configure("attack", foreground="red")

        tk.Button(frame, text="Start Detection", command=self.start_sniffing).pack(pady=5)
        tk.Button(frame, text="Stop Detection", command=self.stop_sniffing_func).pack(pady=5)

        self.current_frame = frame

    def extract_features(self, pkt):
        global flow_data
        if not pkt.haslayer(IP) or not pkt.haslayer(TCP):
            return None
        ip, tcp, now = pkt[IP], pkt[TCP], time.time()
        if flow_data["start_time"] is None:
            flow_data["start_time"] = now
        flow_duration = now - flow_data["start_time"]
        if flow_data["packet_count"] > 0:
            flow_data["iat_times"].append(now - flow_data["last_pkt_time"])
        flow_data["last_pkt_time"] = now
        flow_data["packet_lengths"].append(len(pkt))
        flow_data["packet_count"] += 1
        if ip.dst == pkt[IP].src:
            flow_data["bwd_packet_count"] += 1
        bwd_packet_rate = flow_data["bwd_packet_count"] / flow_duration if flow_duration > 0 else 0
        min_pkt_length = min(flow_data["packet_lengths"]) if flow_data["packet_lengths"] else 0
        features = [
            1 if tcp.flags & 0x08 else 0, bwd_packet_rate, tcp.window,
            sum(flow_data["iat_times"]), 1 if tcp.flags & 0x20 else 0,
            flow_duration, sum(flow_data["iat_times"]), 1 if tcp.flags & 0x10 else 0,
            min_pkt_length, min_pkt_length
        ]
        return np.array(features).reshape(1, -1)

    def predict_packet(self, pkt):
        if stop_sniffing:
            return True
        features = self.extract_features(pkt)
        if features is None:
            return
        scaled = scaler.transform(features)
        prediction = model.predict(scaled)[0]
        if prediction == 1:
            self.real_output.insert(tk.END, f"Prediction: Anomaly Detected\n", "attack")
        else:
            self.real_output.insert(tk.END, f"Prediction: Benign\n")
        self.real_output.see(tk.END)

    def start_sniffing(self):
        global stop_sniffing
        stop_sniffing = False
        self.real_output.insert(tk.END, "Real-time sniffing started...\n")
        threading.Thread(target=lambda: sniff(prn=self.predict_packet, filter="tcp", store=0)).start()

    def stop_sniffing_func(self):
        global stop_sniffing
        stop_sniffing = True
        self.real_output.insert(tk.END, "Sniffing stopped.\n")

# === Start GUI ===
if __name__ == "__main__":
    root = tk.Tk()
    app = AnomalyApp(root)
    root.mainloop()


FileNotFoundError: [Errno 2] No such file or directory: 'network_intrusion_model2.pkl'